<h1>Hard level</h1>

Имеем два случая, кольцо целых чисел и кольцо полиномов от одной переменной. На вход подается список, пораждающий идеал.
Нужно определить является ли он главным и если да, то найти порождающий элемент/полином.

In [3]:
from typing import Union, List

In [4]:
class RingElement:
    """Базовый класс для элементов колец: целых чисел и полиномов."""
    def __init__(self, data: Union[int, List[float]]):
        """
        data:
            - если это целое число → элемент кольца Z;
            - если это список коэффициентов [a0, a1, ..., an] → полином a0 + a1*x + ... + an*x^n.
        """
        self.data = data
        self.is_polynomial = isinstance(data, list)
    
    def __repr__(self) -> str:
        if self.is_polynomial:
            terms = [f"{c}*x^{i}" if i > 0 else str(c) for i, c in enumerate(self.data) if c != 0]
            return " + ".join(terms) if terms else "0"
        return str(self.data)
    
    def is_zero(self) -> bool:
        """Проверяет, является ли элемент нулевым."""
        if self.is_polynomial:
            return len(self.data) == 1 and abs(self.data[0]) < 1e-10
        return self.data == 0


In [ ]:
EPS = 10e-10


def int_gcd(a: int, b: int) -> int:
    """Классический алгоритм Евклида для целых чисел"""
    a, b = abs(a), abs(b)
    if b == 0:
        return a
    return int_gcd(b, a % b)


def polynomial_gcd(p1, p2):
    """
    Алгоритм Евклида для полиномов
        p1, p2 - объекты RingElement (полиномы)
    """
    if p2.is_zero():
        return p1
    
    # Получаем коэффициенты
    dividend = p1.data.copy()
    divisor = p2.data.copy()
    
    # Выполняем деление с остатком
    while len(dividend) >= len(divisor) and not (len(dividend) == 1 and abs(dividend[0]) < EPS):
        if len(divisor) == 1 and abs(divisor[0]) < EPS:
            break
            
        # Вычисляем коэффициент
        degree_diff = len(dividend) - len(divisor)
        factor = dividend[-1] / divisor[-1]
        
        # Вычитаем
        for i in range(len(divisor)):
            idx = degree_diff + i
            if idx < len(dividend):
                dividend[idx] -= factor * divisor[i]
        
        # Нормализуем
        while len(dividend) > 1 and abs(dividend[-1]) < EPS:
            dividend.pop()

    # Создаем остаток
    remainder_coeffs = dividend if dividend else [0.0]
    remainder = RingElement(remainder_coeffs)
    
    # Если остаток нулевой
    if remainder.is_zero():
        if len(p2.data) == 1:
            return RingElement([1.0])
        return p2
    
    # Рекурсивно продолжаем
    return polynomial_gcd(p2, remainder)


def gcd_ring_elements(elements: List[RingElement]) -> RingElement:
    """
    Возвращает порождающий главного идеала, порожденного заданными элементами.
        - Для Z: НОД целых чисел.
        - Для K[x]: НОД полиномов (с коэффициентами в поле K).
    """
    # Проверяем, что все элементы одного типа
    first_type = elements[0].is_polynomial
    for elem in elements:
        if elem.is_polynomial != first_type:
            raise ValueError("Все элементы должны быть одного типа (целые числа или полиномы)")
    
    # Фильтруем нулевые элементы
    elems = [elem for elem in elements if not elem.is_zero()]
    
    if not elems:
        return RingElement(0) if not first_type else RingElement([0.0])
    
    # Начинаем с первого ненулевого элемента
    result = elems[0]
    print(elems)
    
    for elem in elems[1:]:
        if first_type:
            # Случай полиномов
            result = polynomial_gcd(result, elem)
        else:
            # Случай целых чисел
            result = RingElement(int_gcd(result.data, elem.data))
    
    return result


In [ ]:
# Примеры использования для полиномов

print("Полиномы высокой степени с общим множителем")
poly1 = RingElement([-1, 1, 2, -2, -1, 1])  # x^5 - x^4 - 2x^3 + 2x^2 + x - 1
poly2 = RingElement([1, 0, -2, 0, 1])       # x^4 - 2x^2 + 1
gcd1 = gcd_ring_elements([poly1, poly2])
print(f"f(x) = {poly1}")
print(f"g(x) = {poly2}")
print(f"НОД(f, g) = {gcd1}")

print("Полиномы с комплексными корнями")
poly3 = RingElement([1, 0, 2, 0, 1])        # x^4 + 2x^2 + 1
poly4 = RingElement([1, 1, 1, 1])           # x^3 + x^2 + x + 1
gcd2 = gcd_ring_elements([poly3, poly4])
print(f"f(x) = {poly3}")
print(f"g(x) = {poly4}")
print(f"НОД(f, g) = {gcd2}")

print("Три полинома с последовательными общими делителями")
poly5 = RingElement([3, -2, -3, 2])         # 2x^3 - 3x^2 - 2x + 3
poly6 = RingElement([3, -8, 4])             # 4x^2 - 8x + 3
poly7 = RingElement([3, -5, 2])             # 2x^2 - 5x + 3
gcd3 = gcd_ring_elements([poly5, poly6, poly7])
print(f"f(x) = {poly5}")
print(f"g(x) = {poly6}")
print(f"h(x) = {poly7}")
print(f"НОД(f, g, h) = {gcd3}")

print("Взаимно простые полиномы")
poly8 = RingElement([1, -2, 0, 1])          # x^3 - 2x + 1
poly9 = RingElement([1, 1, 1])              # x^2 + x + 1
gcd4 = gcd_ring_elements([poly8, poly9])
print(f"f(x) = {poly8}")
print(f"g(x) = {poly9}")
print(f"НОД(f, g) = {gcd4}")

print("Полиномы с кратными корнями")
poly10 = RingElement([1, -4, 6, -4, 1])     # x^4 - 4x^3 + 6x^2 - 4x + 1
poly11 = RingElement([-1, 3, -3, 1])        # x^3 - 3x^2 + 3x - 1
gcd5 = gcd_ring_elements([poly10, poly11])
print(f"f(x) = {poly10}")
print(f"g(x) = {poly11}")
print(f"НОД(f, g) = {gcd5}")

print("Полиномы с дробными коэффициентами")
poly12 = RingElement([-0.5, 1.5, -1.5, 0.5]) # 0.5x^3 - 1.5x^2 + 1.5x - 0.5
poly13 = RingElement([1, -2, 1])             # x^2 - 2x + 1
gcd6 = gcd_ring_elements([poly12, poly13])
print(f"f(x) = {poly12}")
print(f"g(x) = {poly13}")
print(f"НОД(f, g) = {gcd6}")

print("Много полиномов с разными степенями")
poly14 = RingElement([2, 0, 3, 0, 2])       # 2x^4 + 3x^2 + 2 = (x^2+1)(2x^2+1)
poly15 = RingElement([1, 1, 1, 1])          # x^3 + x^2 + x + 1 = (x^2+1)(x+1)
poly16 = RingElement([0, -1, 0, -1])        # -x^3 - x = -x(x^2+1)
poly17 = RingElement([5, 0, 5])             # 5x^2 + 5 = 5(x^2+1)
gcd7 = gcd_ring_elements([poly14, poly15, poly16, poly17])
print(f"f1(x) = {poly14}")
print(f"f2(x) = {poly15}")
print(f"f3(x) = {poly16}")
print(f"f4(x) = {poly17}")
print(f"НОД(f1, f2, f3, f4) = {gcd7}")

print("Полиномы с числовым НОД")
poly18 = RingElement([4, 11, 6])            # 6x^2 + 11x + 4
poly19 = RingElement([3, 8, 4])             # 4x^2 + 8x + 3
gcd8 = gcd_ring_elements([poly18, poly19])
print(f"f(x) = {poly18}")
print(f"g(x) = {poly19}")
print(f"НОД(f, g) = {gcd8}")


Пример 1: Полиномы высокой степени с общим множителем
[-1 + 1*x^1 + 2*x^2 + -2*x^3 + -1*x^4 + 1*x^5, 1 + -2*x^2 + 1*x^4]
f(x) = -1 + 1*x^1 + 2*x^2 + -2*x^3 + -1*x^4 + 1*x^5
g(x) = 1 + -2*x^2 + 1*x^4
НОД(f, g) = 1 + -2*x^2 + 1*x^4
Пример 2: Полиномы с комплексными корнями
[1 + 2*x^2 + 1*x^4, 1 + 1*x^1 + 1*x^2 + 1*x^3]
f(x) = 1 + 2*x^2 + 1*x^4
g(x) = 1 + 1*x^1 + 1*x^2 + 1*x^3
НОД(f, g) = 2.0 + 2.0*x^2
Пример 3: Три полинома с последовательными общими делителями
[3 + -2*x^1 + -3*x^2 + 2*x^3, 3 + -8*x^1 + 4*x^2, 3 + -5*x^1 + 2*x^2]
f(x) = 3 + -2*x^1 + -3*x^2 + 2*x^3
g(x) = 3 + -8*x^1 + 4*x^2
h(x) = 3 + -5*x^1 + 2*x^2
НОД(f, g, h) = 2.25 + -1.5*x^1
Пример 4: Взаимно простые полиномы
[1 + -2*x^1 + 1*x^3, 1 + 1*x^1 + 1*x^2]
f(x) = 1 + -2*x^1 + 1*x^3
g(x) = 1 + 1*x^1 + 1*x^2
НОД(f, g) = 1.0
Пример 5: Полиномы с кратными корнями
[1 + -4*x^1 + 6*x^2 + -4*x^3 + 1*x^4, -1 + 3*x^1 + -3*x^2 + 1*x^3]
f(x) = 1 + -4*x^1 + 6*x^2 + -4*x^3 + 1*x^4
g(x) = -1 + 3*x^1 + -3*x^2 + 1*x^3
НОД(f, g) = -1 + 3*x^1 